#### Azure Batch Shipyard devito 

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
not_shared_dir_list = ['.', 'not_shared']
DOTENV_FILE_PATH = not_shared_dir_list + ['general.env']
SIBLING_DOCKER_DOTENV_FILE_PATH = not_shared_dir_list + ['sibling_docker.env']

SSH_KEYS_FILE_PATH = not_shared_dir_list + ['azuredevitogatechssh']
ssh_key_file_name = 'id_rsa' #'devito-slim-key-rsa'

docker_file_location = 'docker_files'
docker_build_location = 'docker_build'
docker_build_no_cache = ''  # '--no-cache' # or '' #

control_plane_docker_image_name = "azure_batch_shipyard_devito"
control_plane_docker_image_tag = ""

azure_batch_shipyard_devito_base_image_name = 'devito_azure_base:v1.0'
azure_batch_shipyard_devito_tti_image_name = 'devito_azure_tti:v1.0'

az_cli_container = "azure_cli_container01"
signed_in_az_cli_image = 'new_az_cli_image'


use_ACR_and_dockerhub = True

In [3]:
import sys, os, pathlib, shutil
import platform,  dotenv
import subprocess

In [4]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1091-azure-x86_64-with-debian-10.3'

'/workspace/notebooks'

In [5]:
def create_dot_env_file(dotenv_file_path_list):
    created_dotenv_file_path = os.path.join(*(dotenv_file_path_list)) 
    os.makedirs(os.path.join(*(dotenv_file_path_list[:-1])), exist_ok=True)
    pathlib.Path(created_dotenv_file_path).touch()
    return created_dotenv_file_path

dotenv_file_path = create_dot_env_file(DOTENV_FILE_PATH)
sibling_docker_dotenv_file_path = create_dot_env_file(SIBLING_DOCKER_DOTENV_FILE_PATH)

# # show .env file path
# !pwd
dotenv_file_path
sibling_docker_dotenv_file_path

'./not_shared/general.env'

'./not_shared/sibling_docker.env'

Save your sensitive info

In [6]:
ACR_NAME="fwi01acr"
ACR_PASSWORD=""
ACR_USERNAME="fwi01acr"

dockerhub_login = "georgedockeraccount"
dockerhub_pwd = ""

SUBSCRIPTION_ID=""

docker_container_mount_point = os.getcwd()

In [7]:
# dotenv.set_key(dotenv_file_path, 'ACR_NAME', ACR_NAME)
# dotenv.set_key(dotenv_file_path, 'ACR_PASSWORD', ACR_PASSWORD)
# dotenv.set_key(dotenv_file_path, 'ACR_USERNAME', ACR_USERNAME)

# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_LOGIN', dockerhub_login)
# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_PWD', dockerhub_pwd)

# dotenv.set_key(dotenv_file_path, 'SUBSCRIPTION_ID', SUBSCRIPTION_ID)

# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_name', control_plane_docker_image_name)
# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_tag', control_plane_docker_image_tag)

# dotenv.set_key(sibling_docker_dotenv_file_path, 'DOCKER_CONTAINER_MOUNT_POINT', docker_container_mount_point)

In [8]:
dotenv.load_dotenv(dotenv_path=dotenv_file_path)
docker_file_location = os.path.join(*(os.getcwd(), docker_file_location))

docker_file_name = 'Dockerfile'+ '_' + os.getenv('control_plane_docker_image_name')

docker_image_name_only = os.getenv('control_plane_docker_image_name')
image_version = os.getenv('control_plane_docker_image_tag')
if image_version!="":
    docker_file_name = docker_file_name +'_'+ image_version
    docker_image_name_only = docker_image_name_only +':'+ image_version

# docker_repo_name = os.getenv('DOCKERHUB_LOGIN') # or  
docker_repo_name = os.getenv('ACR_NAME')+'.azurecr.io' 
docker_image_name = docker_repo_name + '/' + docker_image_name_only


docker_file_dir = os.path.join(*([os.getcwd(), docker_file_location]))
os.makedirs(docker_file_dir, exist_ok=True)
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))

docker_build_dir = os.path.join(*([os.getcwd(), docker_build_location]))
os.makedirs(docker_build_dir, exist_ok=True)

ssh_keys_file_dir = os.path.join(*([os.getcwd()]+ SSH_KEYS_FILE_PATH))
os.makedirs(ssh_keys_file_dir, exist_ok=True)

docker_image_name
docker_file_dir
docker_file_path
docker_build_dir
ssh_keys_file_dir

True

'fwi01acr.azurecr.io/azure_batch_shipyard_devito'

'/workspace/notebooks/docker_files'

'/workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito'

'/workspace/notebooks/docker_build'

'/workspace/notebooks/./not_shared/azuredevitogatechssh'

In [9]:
dotenv.load_dotenv(dotenv_path=sibling_docker_dotenv_file_path)
os.getenv('DOCKER_CONTAINER_MOUNT_POINT')

True

'/datadrive01/prj/Azure2019/notebooks'

###### Generate local ssh keys (if needed)

In [10]:
ssh_key_name_full = os.path.join(*([ssh_keys_file_dir]+ [ssh_key_file_name])) 
ssh_key_name_full

pub_ssh_key_name_full = ssh_key_name_full+'.pub'
! ls -la $pub_ssh_key_name_full 

'/workspace/notebooks/./not_shared/azuredevitogatechssh/id_rsa'

-rwxrwxrwx 1 root root 743 Mar 13 22:55 /workspace/notebooks/./not_shared/azuredevitogatechssh/id_rsa.pub


In [11]:
if not (os.path.exists(ssh_key_name_full)):
    print('No ssh key ('+ssh_key_name_full+') found, wil generate a new pair ')
    cli_command = 'ssh-keygen -t rsa -b 4096 -f '+ssh_key_name_full + ' -q -N ""'
    cli_command
    !$cli_command
else:
    print('Found ssh key ('+ssh_key_name_full+'), will use it')  

!ls -la $ssh_keys_file_dir    

Found ssh key (/workspace/notebooks/./not_shared/azuredevitogatechssh/id_rsa), will use it
total 28
drwxrwxrwx 3 root root 4096 Mar 23 22:42 .
drwxrwxrwx 6 1003 1003 4096 Jul  1 02:18 ..
drwxrwxrwx 2 root root 4096 Mar 13 22:29 .ipynb_checkpoints
-rw-rw-r-- 1 1003 1003 1823 Mar 23 22:42 azuredevitogatechsshkey
-rw-rw-r-- 1 1003 1003  399 Mar 23 22:42 azuredevitogatechsshkey.pub
-rwxrwxrwx 1 root root 3381 Mar 13 22:55 id_rsa
-rwxrwxrwx 1 root root  743 Mar 13 22:55 id_rsa.pub


##### Pull existing docker image from ACR, if available
[mcr.microsoft.com/azure-cli](https://github.com/Azure/azure-cli/blob/dev/Dockerfile) is based on alpine, so we need to install docker inside it

In [12]:
!docker rm -f $az_cli_container

Error: No such container: azure_cli_container01


In [13]:
cli_base_command='docker run '+ \
'-it '+ \
'--name ' + az_cli_container+ ' ' + \
'mcr.microsoft.com/azure-cli '+ \
'/bin/bash  -c ' 
internal_command = '"'+ \
'apk update && apk add --update --no-cache  openrc docker-cli docker; '+ \
'rc-update add docker boot; '+ \
' az login; '+ \
' az account set --subscription '+os.getenv('SUBSCRIPTION_ID')+ '; '+ \
' az account list -o table; '+\
'"'

cli_command = cli_base_command+internal_command

cli_command
!$cli_command

'docker run -it --name azure_cli_container01 mcr.microsoft.com/azure-cli /bin/bash  -c "apk update && apk add --update --no-cache  openrc docker-cli docker; rc-update add docker boot;  az login;  az account set --subscription 789908e0-5fc2-4c4d-b5f5-9764b0d602b3;  az account list -o table; "'

Unable to find image 'mcr.microsoft.com/azure-cli:latest' locally
latest: Pulling from azure-cli

e7a5bc2a: Pulling fs layer 
cd19a4e3: Pulling fs layer 
815b6f8f: Pulling fs layer 
981c6bd3: Pulling fs layer 
7f8946aa: Pulling fs layer 
80f66e5d: Pulling fs layer 
5b4eaaf0: Pulling fs layer 
c461a1a3: Pulling fs layer 
09debd14: Pulling fs layer 
30c15fc6: Pulling fs layer 
Digest: sha256:20fc871669309ed059e97686b85543b6078fd648ee01be784586476f882d3849K
Status: Downloaded newer image for mcr.microsoft.com/azure-cli:latest
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/community/x86_64/APKINDEX.tar.gz
v3.11.6-98-gc363fd80c8 [http://dl-cdn.alpinelinux.org/alpine/v3.11/main]
v3.11.6-96-gc0aa9d3d49 [http://dl-cdn.alpinelinux.org/alpine/v3.11/community]
OK: 11275 distinct packages available
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.

7 47% ███████████████████████████████████                                       87 48% ███████████████████████████████████                                       87 48% ████████████████████████████████████                                      87 49% ████████████████████████████████████                                      87 50% █████████████████████████████████████                                     87 51% ██████████████████████████████████████                                    87 52% ██████████████████████████████████████                                    87 52% ███████████████████████████████████████                                   87 53% ███████████████████████████████████████                                   87 54% ████████████████████████████████████████                                  87 55% █████████████████████████████████████████                                 87 56% █████████████████████████████████████████                                 87 5

Name                               CloudName    SubscriptionId                        State    IsDefault
---------------------------------  -----------  ------------------------------------  -------  -----------
Visual Studio Ultimate with MSDN   AzureCloud   fe4e0535-d933-4159-8440-c2908bfc4b05  Enabled  False
AnE.ExP.Production                 AzureCloud   dad45786-32e5-4ef3-b90e-8e0838fbadb6  Enabled  False
Epic-Edge-ES-ReproVMLab            AzureCloud   333e402a-65a0-45a9-8e23-867ca146c290  Enabled  False
Epic-Edge-ES-MergeResolver         AzureCloud   482e1993-01d4-4b16-bff4-1866929176a1  Enabled  False
Core-ES-BranchManagement           AzureCloud   5b4328b8-7b74-4d86-86fd-f2c35f2790fb  Enabled  False
Cosmos_WDG_Core_BnB_100292         AzureCloud   e01de573-132a-42ac-9ee2-f9dea9dd2717  Enabled  False
MSRCSUPP Non-Express Prod CC70550  AzureCloud   4d747266-6ac4-4929-89e2-4cfe9a71858b  Enabled  False
TScience                           AzureCloud   9ec1d932-0f3f-486c-acc6-e7d78b358

In [14]:
# mcr.microsoft.com/azure-cli container was sarted without rm option so it should show up as stopped
!docker container ls -a

CONTAINER ID        IMAGE                         COMMAND                  CREATED             STATUS                    PORTS                     NAMES
f66381381b6e        mcr.microsoft.com/azure-cli   "/bin/bash -c 'apk u…"   4 minutes ago       Exited (0) 1 second ago                             azure_cli_container01
3121d7b06d8e        continuumio/miniconda3        "/bin/bash -c 'ls -l…"   6 minutes ago       Up 6 minutes              0.0.0.0:10003->8888/tcp   miniconda3_container03


In [15]:
!docker commit $az_cli_container $signed_in_az_cli_image
!docker rm -f $az_cli_container

sha256:01b32aff6373f905573d943022cf2d5d028615d66d509d2be96bbc635b4323c0
azure_cli_container01


In [16]:
cli_base_command=' docker run ' +\
'-it '+\
'--name ' + az_cli_container+ ' ' + \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
signed_in_az_cli_image+ ' '+\
'/bin/bash  -c ' 

internal_command = '"'+ \
' : az account list -o table; '+\
'az acr login --name '+os.getenv('ACR_NAME')+ \
' --username '+os.getenv('ACR_USERNAME')+ \
' --password ' + os.getenv('ACR_PASSWORD')+'; '+\
'docker pull '+docker_image_name+'; '+\
'"'

cli_command = cli_base_command+internal_command

# cli_command
!$cli_command

Login Succeeded
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store
Using default tag: latest
latest: Pulling from azure_batch_shipyard_devito

c4f90ab0: Pulling fs layer 
b99eeaa9: Pulling fs layer 
ca8e8768: Pulling fs layer 
Digest: sha256:91f66457cd47c3ba5e9ffaa1ec0614f9e12ba8c86911717438f5716b39e0440d[3A
Status: Downloaded newer image for fwi01acr.azurecr.io/azure_batch_shipyard_devito:latest
fwi01acr.azurecr.io/azure_batch_shipyard_devito:latest


Dockerfile references:  
https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile  
https://github.com/matsuu/docker-azure-cli/blob/master/Dockerfile  
https://github.com/Azure/azure-cli/blob/dev/Dockerfile  
https://github.com/Azure/azure-functions-core-tools#other-linux-distributions  

In [17]:
%%writefile $docker_file_path 

# https://hub.docker.com/r/microsoft/azure-cli/dockerfile   
FROM debian:latest 
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile
# https://github.com/Azure/batch-shipyard/blob/master/cargo/Dockerfile
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    python3 python3-pip python3-setuptools \
    ca-certificates curl apt-transport-https lsb-release gnupg openssh-client openssh-server \
    zip unzip \
    wget rsync git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*   && \
    git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard && \
    cd /opt/batch-shipyard && \
    rm -rf .git .github .vsts && \
    rm -f .git* .travis.yml *.yml install*  && \
    python3 -m pip install --no-cache-dir --upgrade pip  && \
    pip3 install --no-cache-dir -r requirements.txt  && \
    pip3 install --no-cache-dir --no-deps -r req_nodeps.txt && \
    python3 -m compileall -f /opt/batch-shipyard/shipyard.py /opt/batch-shipyard/convoy 
# ENTRYPOINT ["/opt/batch-shipyard/shipyard.py"]

# https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-apt?view=azure-cli-latest#no-package-for-your-distribution
# ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $(lsb_release -sc) main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 

# https://github.com/Azure/azure-functions-core-tools
ENV AzFunctionsCoreTools_DIR /azure-functions-cli
RUN mkdir /tmp/AzFunctionsCoreTools3 && \
    mkdir -p $AzFunctionsCoreTools_DIR && \
    cd /tmp/AzFunctionsCoreTools3 && \
    wget --quiet --no-check-certificate \
    https://github.com/Azure/azure-functions-core-tools/releases/download/2.7.2100/Azure.Functions.Cli.linux-x64.2.7.2100.zip  && \
    unzip  -qq -d $AzFunctionsCoreTools_DIR Azure.Functions.Cli.linux-x64.*.zip && \
    rm -f Azure.Functions.Cli.linux-x64.*.zip && \
    cd $AzFunctionsCoreTools_DIR && \
    chmod +x func
ENV PATH=$AzFunctionsCoreTools_DIR:$PATH    

Overwriting /workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito


In [18]:
cli_command='docker build -t '+ docker_image_name + \
' -f ' + docker_file_path + \
' ' + docker_build_dir + ' ' +\
docker_build_no_cache  + ' ' 
#  ' --target AzureBatchShipyardAzCliAzFunctionsCoreTools'

cli_command
# docker_build_response = ! $cli_command

# docker_build_response[0:5] 
# docker_build_response[-5:]

'docker build -t fwi01acr.azurecr.io/azure_batch_shipyard_devito -f /workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito /workspace/notebooks/docker_build  '

In [19]:
! $cli_command

Sending build context to Docker daemon  3.631kB
Step 1/7 : FROM debian:latest
latest: Pulling from library/debian

Digest: sha256:68f4e2259032a4e6f5035804e64438b52af8dd5889528b305b9059183ea4cd2a[1A
Status: Downloaded newer image for debian:latest
 ---> ae8514941ea4
Step 2/7 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 5684ad85311b
Removing intermediate container 5684ad85311b
 ---> 0f81ed53dc1c
Step 3/7 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     python3 python3-pip python3-setuptools     ca-certificates curl apt-transport-https lsb-release gnupg openssh-client openssh-server     zip unzip     wget rsync git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*   &&     git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard &&     cd /opt/batch-shipyard &&     rm -rf .git .github .vsts &&     rm -f .git* .travis.yml *.yml install*  &&     python3 -m pip install --no-cache-dir --upgrade pip  &&     

Get:38 http://deb.debian.org/debian buster/main amd64 openssl amd64 1.1.1d-0+deb10u3 [844 kB]
Get:39 http://deb.debian.org/debian buster-updates/main amd64 ca-certificates all 20200601~deb10u1 [158 kB]
Get:40 http://deb.debian.org/debian buster/main amd64 libsasl2-modules-db amd64 2.1.27+dfsg-1+deb10u1 [69.1 kB]
Get:41 http://deb.debian.org/debian buster/main amd64 libsasl2-2 amd64 2.1.27+dfsg-1+deb10u1 [106 kB]
Get:42 http://deb.debian.org/debian buster/main amd64 libldap-common all 2.4.47+dfsg-3+deb10u2 [89.7 kB]
Get:43 http://deb.debian.org/debian buster/main amd64 libldap-2.4-2 amd64 2.4.47+dfsg-3+deb10u2 [224 kB]
Get:44 http://deb.debian.org/debian buster/main amd64 libnghttp2-14 amd64 1.36.0-2+deb10u1 [85.0 kB]
Get:45 http://deb.debian.org/debian buster/main amd64 librtmp1 amd64 2.4+20151223.gitfa8646d.1-2 [60.5 kB]
Get:46 http://deb.debian.org/debian buster/main amd64 libssh2-1 amd64 1.8.0-2.1 [140 kB]
Get:47 http://deb.debian.org/debian buster/main amd64 libcurl4 amd64 7.64.0-4

Selecting previously unselected package libprocps7:amd64.
Preparing to unpack .../02-libprocps7_2%3a3.3.15-2_amd64.deb ...
Unpacking libprocps7:amd64 (2:3.3.15-2) ...
Selecting previously unselected package lsb-base.
Preparing to unpack .../03-lsb-base_10.2019051400_all.deb ...
Unpacking lsb-base (10.2019051400) ...
Selecting previously unselected package procps.
Preparing to unpack .../04-procps_2%3a3.3.15-2_amd64.deb ...
Unpacking procps (2:3.3.15-2) ...
Selecting previously unselected package sensible-utils.
Preparing to unpack .../05-sensible-utils_0.0.12_all.deb ...
Unpacking sensible-utils (0.0.12) ...
Selecting previously unselected package libbsd0:amd64.
Preparing to unpack .../06-libbsd0_0.9.1-2_amd64.deb ...
Unpacking libbsd0:amd64 (0.9.1-2) ...
Selecting previously unselected package libedit2:amd64.
Preparing to unpack .../07-libedit2_3.1-20181209-1_amd64.deb ...
Unpacking libedit2:amd64 (3.1-20181209-1) ...
Selecting previously unselected package libkeyutils1:amd64.
Prepari

Selecting previously unselected package openssh-server.
Preparing to unpack .../53-openssh-server_1%3a7.9p1-10+deb10u2_amd64.deb ...
Unpacking openssh-server (1:7.9p1-10+deb10u2) ...
Selecting previously unselected package python-pip-whl.
Preparing to unpack .../54-python-pip-whl_18.1-5_all.deb ...
Unpacking python-pip-whl (18.1-5) ...
Selecting previously unselected package python3-lib2to3.
Preparing to unpack .../55-python3-lib2to3_3.7.3-1_all.deb ...
Unpacking python3-lib2to3 (3.7.3-1) ...
Selecting previously unselected package python3-distutils.
Preparing to unpack .../56-python3-distutils_3.7.3-1_all.deb ...
Unpacking python3-distutils (3.7.3-1) ...
Selecting previously unselected package python3-pip.
Preparing to unpack .../57-python3-pip_18.1-5_all.deb ...
Unpacking python3-pip (18.1-5) ...
Selecting previously unselected package python3-pkg-resources.
Preparing to unpack .../58-python3-pkg-resources_40.8.0-1_all.deb ...
Unpacking python3-pkg-resources (40.8.0-1) ...
Selecting 

Using legacy setup.py install for future, since package 'wheel' is not installed.
Using legacy setup.py install for bitstring, since package 'wheel' is not installed.
ERROR: blobxfer 1.9.4 has requirement cryptography~=2.8, but you'll have cryptography 3.0 which is incompatible.
    Running setup.py install for future: started
    Running setup.py install for future: finished with status 'done'
    Running setup.py install for bitstring: started
    Running setup.py install for bitstring: finished with status 'done'
Compiling '/opt/batch-shipyard/shipyard.py'...
Listing '/opt/batch-shipyard/convoy'...
Compiling '/opt/batch-shipyard/convoy/__init__.py'...
Compiling '/opt/batch-shipyard/convoy/aad.py'...
Compiling '/opt/batch-shipyard/convoy/autoscale.py'...
Compiling '/opt/batch-shipyard/convoy/batch.py'...
Compiling '/opt/batch-shipyard/convoy/clients.py'...
Compiling '/opt/batch-shipyard/convoy/crypto.py'...
Compiling '/opt/batch-shipyard/convoy/data.py'...
Compiling '/opt/batch-shipy

debconf: delaying package configuration, since apt-utils is not installed
Fetched 49.5 MB in 1s (54.9 MB/s)
Selecting previously unselected package azure-cli.
(Reading database ... 11808 files and directories currently installed.)
Preparing to unpack .../azure-cli_2.9.1-1~buster_all.deb ...
Unpacking azure-cli (2.9.1-1~buster) ...
Setting up azure-cli (2.9.1-1~buster) ...
Removing intermediate container 70ec15647758
 ---> e5e306c7fec8
Step 5/7 : ENV AzFunctionsCoreTools_DIR /azure-functions-cli
 ---> Running in cf35569b0b5c
Removing intermediate container cf35569b0b5c
 ---> 8fa08f8408fd
Step 6/7 : RUN mkdir /tmp/AzFunctionsCoreTools3 &&     mkdir -p $AzFunctionsCoreTools_DIR &&     cd /tmp/AzFunctionsCoreTools3 &&     wget --quiet --no-check-certificate     https://github.com/Azure/azure-functions-core-tools/releases/download/2.7.2100/Azure.Functions.Cli.linux-x64.2.7.2100.zip  &&     unzip  -qq -d $AzFunctionsCoreTools_DIR Azure.Functions.Cli.linux-x64.*.zip &&     rm -f Azure.Functio

##### Build base docker image
 - use control plane docker image just built above
 - build azure_batch_shipyard_devito_base_image_name defined by src/AzureBatch/docker/base_image/Dockerfile
 - creates public and private ssh keys in /root/.ssh/ in azure_batch_shipyard_devito_base_image_name docker image
 - cp /root/.ssh/id_rsa.pub /root/.ssh/authorized_keys

In [20]:
# these two paths point to the same location, one inside this control plane container and one on host VM
os.path.join(*([os.getcwd()]+ SSH_KEYS_FILE_PATH))
os.path.join(*([os.getenv('DOCKER_CONTAINER_MOUNT_POINT')]+ SSH_KEYS_FILE_PATH))

'/workspace/notebooks/./not_shared/azuredevitogatechssh'

'/datadrive01/prj/Azure2019/notebooks/./not_shared/azuredevitogatechssh'

In [21]:
cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v '+os.path.join(*([os.getenv('DOCKER_CONTAINER_MOUNT_POINT')]+ SSH_KEYS_FILE_PATH))+':/root/.ssh:ro '+ \
'-v /usr/bin/docker:/usr/bin/docker '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
docker_image_name + ' '+ \
'/bin/bash -c ' 

image_name = azure_batch_shipyard_devito_base_image_name
sibling_docker_build_command = '"cd /workspace/src/AzureBatch/docker/base_image/ &&  docker build -t '+\
image_name + ' ."'
# sibling_docker_build_command = '"ls -la /root/.ssh/; cat /root/.ssh/id_rsa.pub"'
cli_command = cli_base_command+sibling_docker_build_command
cli_command

!$cli_command

'docker run -it --rm --name azure-cli_container01 -v /datadrive01/prj/Azure2019/notebooks/../:/workspace:rw -v /datadrive01/prj/Azure2019/notebooks/./not_shared/azuredevitogatechssh:/root/.ssh:ro -v /usr/bin/docker:/usr/bin/docker -v /var/run/docker.sock:/var/run/docker.sock fwi01acr.azurecr.io/azure_batch_shipyard_devito /bin/bash -c "cd /workspace/src/AzureBatch/docker/base_image/ &&  docker build -t devito_azure_base:v1.0 ."'

Sending build context to Docker daemon  4.096kB
Step 1/9 : FROM ubuntu:18.04
18.04: Pulling from library/ubuntu

5296b23d: Pulling fs layer 
2a4a0f38: Pulling fs layer 
a3b32996: Pulling fs layer 
Digest: sha256:e5b0b89c846690afe2ce325ac6c6bc3d686219cfa82166fc75c812c1011f0803[4A
Status: Downloaded newer image for ubuntu:18.04
 ---> d27b9ffc5667
Step 2/9 : COPY ssh_config /root/.ssh/config
 ---> e58ba499298a
Step 3/9 : RUN apt-get update     && apt-get install -y --no-install-recommends         build-essential ca-certificates wget openssh-client openssh-server         mpich libmpich-dev     && mkdir -p /var/run/sshd     && ssh-keygen -A     && sed -i 's/UsePAM yes/UsePAM no/g' /etc/ssh/sshd_config     && sed -i 's/#PermitRootLogin yes/PermitRootLogin yes/g' /etc/ssh/sshd_config     && sed -i 's/#RSAAuthentication yes/RSAAuthentication yes/g' /etc/ssh/sshd_config     && sed -i 's/#PubkeyAuthentication yes/PubkeyAuthentication yes/g' /etc/ssh/sshd_config     && ssh-keygen -f /root/.ssh/id

Get:25 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libbinutils amd64 2.30-21ubuntu1~18.04.3 [488 kB]
Get:26 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 binutils-x86-64-linux-gnu amd64 2.30-21ubuntu1~18.04.3 [1839 kB]
Get:27 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 binutils amd64 2.30-21ubuntu1~18.04.3 [3388 B]
Get:28 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libc-dev-bin amd64 2.27-3ubuntu1.2 [71.8 kB]
Get:29 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 linux-libc-dev amd64 4.15.0-112.113 [982 kB]
Get:30 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libc6-dev amd64 2.27-3ubuntu1.2 [2585 kB]
Get:31 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gcc-7-base amd64 7.5.0-3ubuntu1~18.04 [18.3 kB]
Get:32 http://archive.ubuntu.com/ubuntu bionic/main amd64 libisl19 amd64 0.19-1 [551 kB]
Get:33 http://archive.ubuntu.com/ubuntu bionic/main amd64 libmpfr6 amd64 4.0.1-1 [243 kB]
Get:34 http://arc

Selecting previously unselected package libkeyutils1:amd64.
Preparing to unpack .../13-libkeyutils1_1.5.9-9.2ubuntu2_amd64.deb ...
Unpacking libkeyutils1:amd64 (1.5.9-9.2ubuntu2) ...
Selecting previously unselected package libkrb5-3:amd64.
Preparing to unpack .../14-libkrb5-3_1.16-2ubuntu0.1_amd64.deb ...
Unpacking libkrb5-3:amd64 (1.16-2ubuntu0.1) ...
Selecting previously unselected package libgssapi-krb5-2:amd64.
Preparing to unpack .../15-libgssapi-krb5-2_1.16-2ubuntu0.1_amd64.deb ...
Unpacking libgssapi-krb5-2:amd64 (1.16-2ubuntu0.1) ...
Selecting previously unselected package libnuma1:amd64.
Preparing to unpack .../16-libnuma1_2.0.11-2.1ubuntu0.1_amd64.deb ...
Unpacking libnuma1:amd64 (2.0.11-2.1ubuntu0.1) ...
Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../17-libpsl5_0.19.1-5build1_amd64.deb ...
Unpacking libpsl5:amd64 (0.19.1-5build1) ...
Selecting previously unselected package libssl1.0.0:amd64.
Preparing to unpack .../18-libssl1.0.0_1.0.2n-1ubunt

Selecting previously unselected package libltdl7:amd64.
Preparing to unpack .../62-libltdl7_2.4.6-2_amd64.deb ...
Unpacking libltdl7:amd64 (2.4.6-2) ...
Selecting previously unselected package libwrap0:amd64.
Preparing to unpack .../63-libwrap0_7.6.q-27_amd64.deb ...
Unpacking libwrap0:amd64 (7.6.q-27) ...
Selecting previously unselected package openssh-sftp-server.
Preparing to unpack .../64-openssh-sftp-server_1%3a7.6p1-4ubuntu0.3_amd64.deb ...
Unpacking openssh-sftp-server (1:7.6p1-4ubuntu0.3) ...
Selecting previously unselected package openssh-server.
Preparing to unpack .../65-openssh-server_1%3a7.6p1-4ubuntu0.3_amd64.deb ...
Unpacking openssh-server (1:7.6p1-4ubuntu0.3) ...
Selecting previously unselected package libhwloc5:amd64.
Preparing to unpack .../66-libhwloc5_1.11.9-1_amd64.deb ...
Unpacking libhwloc5:amd64 (1.11.9-1) ...
Selecting previously unselected package hwloc-nox.
Preparing to unpack .../67-hwloc-nox_1.11.9-1_amd64.deb ...
Unpacking hwloc-nox (1.11.9-1) ...
Selecti

...

Selecting previously unselected package vim.
Preparing to unpack .../38-vim_2%3a8.0.1453-1ubuntu1.3_amd64.deb ...
Unpacking vim (2:8.0.1453-1ubuntu1.3) ...
Setting up readline-common (7.0-3) ...
Setting up git-man (1:2.17.1-1ubuntu0.7) ...
Setting up libexpat1:amd64 (2.2.5-3ubuntu0.2) ...
Setting up less (487-0.1) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up libnghttp2-14:amd64 (1.30.0-1ubuntu1) ...
Setting up mime-support (3.60ubuntu1) ...
Setting up xxd (2:8.0.1453-1ubuntu1.3) ...
Setting up liberror-perl (0.17025-1) ...
Setting up libgpm2:amd64 (1.20.7-5) ...
Setting up libldap-common (2.4.45+dfsg-1ubuntu1.6) ...
Setting up libreadline7:amd64 (7.0-3) ...
Setting up libsasl2-modules-db:amd64 (2.1.27~101-g0780600+dfsg-3ubuntu2.1) ...
Setting up libsasl2-2:amd64 (2.1.27~101-g0780600+dfsg-3ubuntu2.1) ...
Setting up libroken18-heimdal:amd64 (7.5.0+dfsg-1) ...
Set

The following NEW packages will be installed:
  dbus dh-python gir1.2-glib-2.0 libapparmor1 libdbus-1-3 libexpat1-dev
  libgirepository-1.0-1 libglib2.0-0 libglib2.0-data libicu60 libpython3-dev
  libpython3-stdlib libpython3.6-dev libxml2 python-pip-whl python3
  python3-asn1crypto python3-cffi-backend python3-crypto python3-cryptography
  python3-dbus python3-dev python3-distutils python3-gi python3-idna
  python3-keyring python3-keyrings.alt python3-lib2to3 python3-minimal
  python3-pip python3-pkg-resources python3-secretstorage python3-setuptools
  python3-six python3-wheel python3-xdg python3.6 python3.6-dev
  python3.6-minimal shared-mime-info xdg-user-dirs
0 upgraded, 41 newly installed, 0 to remove and 3 not upgraded.
Need to get 61.8 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 python3.6-minimal amd64 3.6.9-1~18.04ubuntu1.1 [1610 kB]
Get:2 http://archive.ubuntu.com/ubuntu b

Selecting previously unselected package shared-mime-info.
Preparing to unpack .../12-shared-mime-info_1.9-2_amd64.deb ...
Unpacking shared-mime-info (1.9-2) ...
Selecting previously unselected package xdg-user-dirs.
Preparing to unpack .../13-xdg-user-dirs_0.17-1ubuntu1_amd64.deb ...
Unpacking xdg-user-dirs (0.17-1ubuntu1) ...
Selecting previously unselected package python3-lib2to3.
Preparing to unpack .../14-python3-lib2to3_3.6.9-1~18.04_all.deb ...
Unpacking python3-lib2to3 (3.6.9-1~18.04) ...
Selecting previously unselected package python3-distutils.
Preparing to unpack .../15-python3-distutils_3.6.9-1~18.04_all.deb ...
Unpacking python3-distutils (3.6.9-1~18.04) ...
Selecting previously unselected package dh-python.
Preparing to unpack .../16-dh-python_3.20180325ubuntu2_all.deb ...
Unpacking dh-python (3.20180325ubuntu2) ...
Selecting previously unselected package libexpat1-dev:amd64.
Preparing to unpack .../17-libexpat1-dev_2.2.5-3ubuntu0.2_amd64.deb ...
Unpacking libexpat1-dev:am

  Created wheel for devito: filename=devito-4.2.2+293.g3726c637-py3-none-any.whl size=383762 sha256=99d5a00b240cb1f4c594394aa28e74ba48b48a0fd6c1faf6b6209dd0bc22d32c
  Stored in directory: /tmp/pip-ephem-wheel-cache-xhfwwb5e/wheels/fa/16/d5/e6ff4407842c8c261bdbc5a422361f1287c005fbb1f847798e
  Created wheel for cgen: filename=cgen-2020.1-py3-none-any.whl size=16549 sha256=8d62de3713c532fd4ab125920abecd23141d49642c29787c5433a12deb18c960
  Stored in directory: /root/.cache/pip/wheels/a2/52/4b/0a2acbb2649d8efa501a8dfe5a0ab01465e6500bbe488ff630
  Created wheel for codepy: filename=codepy-2019.1-py3-none-any.whl size=20579 sha256=c20620365d7a9db9388e5e20bfeb97bc565cbb0e3fc1b7a3d67c6851b1ffdeee
  Stored in directory: /root/.cache/pip/wheels/0b/10/97/62c75da81f3e1ffe0a47a46e033cb7a0545d505591bd0de3a7
  Created wheel for psutil: filename=psutil-5.7.2-cp36-cp36m-linux_x86_64.whl size=287061 sha256=1c8660dceae62abda8bf16921527e30770074cb447670448712a595c59b18d2a
  Stored in directory: /root/.cache

  Created wheel for pytools: filename=pytools-2020.3.1-py2.py3-none-any.whl size=66031 sha256=f908da4454b54695eda907cd8b5b1ff226d40eaf44ea4798b58f01d9fa3555dc
  Stored in directory: /root/.cache/pip/wheels/29/f3/97/6593e46f5e154892eb184f4efe4e698aa6fe9ce80cdff0dcf7
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=45919 sha256=48756e9296827088a703effdc32c85139efc75b2dafe836141820ca580e28061
  Stored in directory: /root/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=5272 sha256=8a9848bdabae93779d755f09394692a905e4ed75f50253803fbe65250a7effd6
  Stored in directory: /root/.cache/pip/wheels/41/11/53/911724983aa48deb94792432e14e518447212dd6c5477d49d3
  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=429299 sha256=8490218c104567f264eeca8c3c158e4997b01fbddfbe76e7948bdd6ad64310dc
  Stored in directory: /root/.cache/pi

##### Build tti docker image
 - use azure_batch_shipyard_devito_base_image_name docker image just built above
 - build azure_batch_shipyard_devito_tti_image_name defined by src/AzureBatch/docker/tti_image/Dockerfile
 - uses /root/.ssh/authorized_keys created above in azure_batch_shipyard_devito_base_image_name docker image

In [22]:
cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v '+os.path.join(*([os.getenv('DOCKER_CONTAINER_MOUNT_POINT')]+ SSH_KEYS_FILE_PATH))+':/root/.ssh:ro '+ \
'-v /usr/bin/docker:/usr/bin/docker '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
azure_batch_shipyard_devito_base_image_name + ' '+ \
'/bin/bash -c ' 

image_name = azure_batch_shipyard_devito_tti_image_name
sibling_docker_build_command = '"cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t '+\
image_name + ' ."'
cli_command = cli_base_command+sibling_docker_build_command
cli_command
!$cli_command

'docker run -it --rm --name azure-cli_container01 -v /datadrive01/prj/Azure2019/notebooks/../:/workspace:rw -v /datadrive01/prj/Azure2019/notebooks/./not_shared/azuredevitogatechssh:/root/.ssh:ro -v /usr/bin/docker:/usr/bin/docker -v /var/run/docker.sock:/var/run/docker.sock devito_azure_base:v1.0 /bin/bash -c "cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t devito_azure_tti:v1.0 ."'

Sending build context to Docker daemon  103.9kB
Step 1/7 : FROM devito_azure_base:v1.0
 ---> dccd2c2b17bc
Step 2/7 : RUN pip3 install azure-storage-blob h5py segyio boto3
 ---> Running in 995fe75e8c0e
Removing intermediate container 995fe75e8c0e
 ---> cb1446c5a93f
Step 3/7 : RUN pip3 install azure-storage-queue
 ---> Running in 58c1ffec7606
Removing intermediate container 58c1ffec7606
 ---> 2f46df2a3238
Step 4/7 : RUN pip3 install sympy==1.5
 ---> Running in 00a611f3d20b
  Attempting uninstall: sympy
    Found existing installation: sympy 1.5.1
    Uninstalling sympy-1.5.1:
      Successfully uninstalled sympy-1.5.1
Removing intermediate container 00a611f3d20b
 ---> 5f0224c42f9f
Step 5/7 : ADD ./tti /app/tti
 ---> 1337e1a82f2a
Step 6/7 : EXPOSE 23
 ---> Running in 3027f48efec3
Removing intermediate container 3027f48efec3
 ---> 183be7b45d6e
Step 7/7 : CMD ["/usr/sbin/sshd", "-D", "-p", "23"]
 ---> Running in 971cb382654d
Removing intermediate container 971cb382654d
 ---> 282e82efc830
Su

##### Use control plane docker image to self push into ACR

In [23]:
full_azure_batch_shipyard_devito_base_image_name=docker_repo_name+ '/' + azure_batch_shipyard_devito_base_image_name
full_azure_batch_shipyard_devito_tti_image_name=docker_repo_name+ '/' + azure_batch_shipyard_devito_tti_image_name

cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
signed_in_az_cli_image + ' '+ \
'/bin/bash -c ' 


internal_command = '"'+ \
'az acr login --name '+os.getenv('ACR_NAME')+ \
' --username '+os.getenv('ACR_USERNAME')+ \
' --password ' + os.getenv('ACR_PASSWORD')+'; '+\
'docker push '+docker_image_name+'; '+\
'docker tag '+ azure_batch_shipyard_devito_base_image_name+' '+ full_azure_batch_shipyard_devito_base_image_name+'; '+\
'docker tag '+ azure_batch_shipyard_devito_tti_image_name+' '+ full_azure_batch_shipyard_devito_tti_image_name+'; '+\
'docker push '+full_azure_batch_shipyard_devito_base_image_name+'; '+\
'docker push '+full_azure_batch_shipyard_devito_tti_image_name+'; '+\
'"'
cli_command = cli_base_command+internal_command
# cli_command
!$cli_command

Login Succeeded
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store
The push refers to repository [fwi01acr.azurecr.io/azure_batch_shipyard_devito]

cc751e65: Preparing 
a5381aad: Preparing 
19122a88: Preparing 
cc751e65: Pushed   532.6MB/530.1MBlatest: digest: sha256:05c11285686f7d3d79ddfeb92b2c345dd190014103e6dcc9c8d3f86b62a8d034 size: 1167
The push refers to repository [fwi01acr.azurecr.io/devito_azure_base]

8f20713e: Preparing 
d99cae12: Preparing 
427a628b: Preparing 
6c3b9886: Preparing 
104e5d91: Preparing 
61e68f74: Preparing 
41bf6dc3: Preparing 
29413313: Preparing 
a7bca5b7: Preparing 
e1f1e531: Preparing 
9d6bc5ec: Preparing 
41bf6dc3: Pushed   302.2MB/297.5MBv1.0: digest: sha256:6837403005ce87843074f1125ae858e32d54436b24b7e4de872d6fd37cb95f50 size: 2839
The push refers to repository [fwi01acr.azurecr.io/devito_

In [24]:
if use_ACR_and_dockerhub:
    control_plane_docker_image_dockerhub_name = os.getenv('DOCKERHUB_LOGIN')+ '/' + docker_image_name_only
    cli_command='docker tag '+ docker_image_name+' '+ control_plane_docker_image_dockerhub_name
    cli_command

    docker_pwd = os.getenv('DOCKERHUB_PWD')
    docker_login = os.getenv('DOCKERHUB_LOGIN')
    !docker login -u=$docker_login -p=$docker_pwd
    !docker push {control_plane_docker_image_dockerhub_name}
 

'docker tag fwi01acr.azurecr.io/azure_batch_shipyard_devito georgedockeraccount/azure_batch_shipyard_devito'

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [docker.io/georgedockeraccount/azure_batch_shipyard_devito]
An image does not exist locally with the tag: georgedockeraccount/azure_batch_shipyard_devito


In [25]:
# !docker image ls
# !docker tag fwi01acr.azurecr.io/azure_batch_shipyard_devito georgedockeraccount/azure_batch_shipyard_devito
# !docker image ls
# !docker push georgedockeraccount/azure_batch_shipyard_devito